In [ ]:
%load_ext autoreload
%autoreload 2

from src.pipeline import run_rag_static
from local_test import load_validation_data
import random

data = load_validation_data("t2t_val.jsonl")
idx = random.randint(0, len(data) - 1)
item = data[idx]
query = item["query"]
iid = item["iid"]
print(item)
# report = run_rag_static(query)

In [1]:
# SPDX-License-Identifier: Apache-2.0

from vllm import LLM

# Sample prompts.
text_1 = "What is the capital of France?"
texts_2 = [
    "The capital of Brazil is Brasilia.", "The capital of France is Paris."
]

# Create an LLM.
# You should pass task="score" for cross-encoder models
model = LLM(
    model="BAAI/bge-reranker-v2-m3",
    task="score",
    enforce_eager=True,
)

# Generate scores. The output is a list of ScoringRequestOutputs.
outputs = model.score(text_1, texts_2)

# Print the outputs.
for text_2, output in zip(texts_2, outputs):
    score = output.outputs.score
    print(f"Pair: {[text_1, text_2]!r} | Score: {score}")

INFO 10-07 16:36:29 [utils.py:233] non-default args: {'task': 'score', 'disable_log_stats': True, 'enforce_eager': True, 'model': 'BAAI/bge-reranker-v2-m3'}
INFO 10-07 16:36:29 [model.py:547] Resolved architecture: XLMRobertaForSequenceClassification
INFO 10-07 16:36:29 [model.py:1730] Downcasting torch.float32 to torch.float16.
INFO 10-07 16:36:29 [model.py:1510] Using max model len 8192
INFO 10-07 16:36:29 [arg_utils.py:1575] (Disabling) chunked prefill by default
INFO 10-07 16:36:29 [arg_utils.py:1578] (Disabling) prefix caching by default
INFO 10-07 16:36:29 [__init__.py:381] Cudagraph is disabled under eager mode
INFO 10-07 16:36:29 [__init__.py:440] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
(EngineCore_DP0 pid=251602) INFO 10-07 16:36:31 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=251602) INFO 10-07 16:36:31 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='BAAI/bge-reranker-v2-m3', specul

(EngineCore_DP0 pid=251602) Process EngineCore_DP0:
(EngineCore_DP0 pid=251602) Traceback (most recent call last):
(EngineCore_DP0 pid=251602)   File "/system/conda/miniconda3/uv/python/cpython-3.12.11-linux-x86_64-gnu/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
(EngineCore_DP0 pid=251602)     self.run()
(EngineCore_DP0 pid=251602)   File "/system/conda/miniconda3/uv/python/cpython-3.12.11-linux-x86_64-gnu/lib/python3.12/multiprocessing/process.py", line 108, in run
(EngineCore_DP0 pid=251602)     self._target(*self._args, **self._kwargs)
(EngineCore_DP0 pid=251602)   File "/teamspace/studios/this_studio/.venv/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 712, in run_engine_core
(EngineCore_DP0 pid=251602)     raise e
(EngineCore_DP0 pid=251602)   File "/teamspace/studios/this_studio/.venv/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 699, in run_engine_core
(EngineCore_DP0 pid=251602)     engine_core = EngineCoreProc(*args, **kwargs)
(

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

In [ ]:
# SPDX-License-Identifier: Apache-2.0
# SPDX-FileCopyrightText: Copyright contributors to the vLLM project
# ruff: noqa: E501
# %pip install vllm
from vllm import LLM

model_name = "Qwen/Qwen3-Reranker-0.6B"

# What is the difference between the official original version and one
# that has been converted into a sequence classification model?
# Qwen3-Reranker is a language model that doing reranker by using the
# logits of "no" and "yes" tokens.
# It needs to computing 151669 tokens logits, making this method extremely
# inefficient, not to mention incompatible with the vllm score API.
# A method for converting the original model into a sequence classification
# model was proposed. See：https://huggingface.co/Qwen/Qwen3-Reranker-0.6B/discussions/3
# Models converted offline using this method can not only be more efficient
# and support the vllm score API, but also make the init parameters more
# concise, for example.
# model = LLM(model="tomaarsen/Qwen3-Reranker-0.6B-seq-cls", task="score")

# If you want to load the official original version, the init parameters are
# as follows.


def get_model() -> LLM:
    """Initializes and returns the LLM model for Qwen3-Reranker."""
    return LLM(
        model=model_name,
        task="score",
        hf_overrides={
            "architectures": ["Qwen3ForSequenceClassification"],
            "classifier_from_token": ["no", "yes"],
            "is_original_qwen3_reranker": True,
        },
    )


# Why do we need hf_overrides for the official original version:
# vllm converts it to Qwen3ForSequenceClassification when loaded for
# better performance.
# - Firstly, we need using `"architectures": ["Qwen3ForSequenceClassification"],`
# to manually route to Qwen3ForSequenceClassification.
# - Then, we will extract the vector corresponding to classifier_from_token
# from lm_head using `"classifier_from_token": ["no", "yes"]`.
# - Third, we will convert these two vectors into one vector.  The use of
# conversion logic is controlled by `using "is_original_qwen3_reranker": True`.

# Please use the query_template and document_template to format the query and
# document for better reranker results.

prefix = '<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be "yes" or "no".<|im_end|>\n<|im_start|>user\n'
suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"

query_template = "{prefix}<Instruct>: {instruction}\n<Query>: {query}\n"
document_template = "<Document>: {doc}{suffix}"


instruction = (
    "Given a web search query, retrieve relevant passages that answer the query"
)

queries = [
    "What is the capital of China?",
    "Explain gravity",
]

documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun.",
]

queries = [
    query_template.format(prefix=prefix, instruction=instruction, query=query)
    for query in queries
]
documents = [document_template.format(doc=doc, suffix=suffix) for doc in documents]

# model = get_model()
model = LLM(model="tomaarsen/Qwen3-Reranker-0.6B-seq-cls", task="score",
max_model_len=512,
gpu_memory_utilization = 0.1,
)
# outputs = model.score(queries, documents)

# print("-" * 30)
# print([output.outputs.score for output in outputs])
# print("-" * 30)


In [ ]:
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun.",
    "It's very dark and high pressure",
    "The bottom if mariana trench is very interesting"
]
outputs = model.score("What is in the mariana trench", documents)
scores = [output.outputs.score for output in outputs]
idxs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=False)

In [4]:
scores = [output.outputs.score for output in outputs]
idxs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=False)

In [ ]:
from src.retriever import retrieve
out = retrieve("How far is the sun from earth")
print(out)